In [229]:
# read categories.txt
lines = sc.textFile("file:///home/cloudera/illi/dm-w1-categories.txt")

In [230]:
lines.count()

77185

In [231]:
# use word count to calculate 1-item frequent and support(count)
words = lines.flatMap(lambda line : line.split(";"))
tuples = words.map(lambda word : (word, 1))
counts = tuples.reduceByKey(lambda a, b: (a + b))
counts.coalesce(1).saveAsTextFile('file:///home/cloudera/illi/count')

In [2]:
# part 1
# read wordcount output file created above and change the format to meet part 1 requests.

file1 = open("/home/cloudera/illi/count/part-00000","r")
file2 = open("/home/cloudera/illi/part1.txt","w")
file3 = open("/home/cloudera/illi/part1-rest.txt","w")

for line in file1:
    line = line.replace("\", ", "', ")
    line = line[1:-2]
    line = line.split("', ")
    support = line[1]
    item = line[0]
    #print(item)
    item = item[1:]
    #file2.write("%s:%s\n" % (support, item))
    if int(support) >= 771:
        file2.write("%s:%s\n" % (support, item))
    else:
        file3.write("%s:%s\n" % (support, item))

file1.close()
file2.close()
file3.close()

In [57]:
# part 2 - 1
# generating

# generate 2-items from frequent 1-items part1.txt
file1 = open("/home/cloudera/illi/part1.txt","r")
file2 = open("/home/cloudera/illi/part2-two-items.txt","w")

items_list = []

for line in file1:
    supp,item = line.split(":")
    item = item[:-1]
    items_list.append(item)
    

import itertools
for subset in itertools.combinations(items_list, 2):
    file2.write("%s;%s\n" % (subset[0], subset[1]))
    
file1.close()
file2.close()

In [58]:
# part 2 - 2
# trim data-  generate a new categories file that has removed infrequent 1-item categories.

file1 = open("/home/cloudera/illi/part1-rest.txt", "r")

infrequent_list = []
for line in file1:
    supp,item = line.split(":")
    infrequent_list.append(item)
    
file2 = open("/home/cloudera/illi/dm-w1-categories.txt", "r")
file3 = open("/home/cloudera/illi/categories-drop-infrequent-1-item.txt", "w")

for line in file2:
    flag = 0
    for item in infrequent_list:
        item = item[:-1]
        if item in line:
            flag = 1
            break;
    if flag == 0:
        file3.write(line)

file1.close()
file2.close()
file3.close()

In [59]:
# remove 1-item transaction as we are working on 2-item and above transactions.
file1 = open("/home/cloudera/illi/categories-drop-infrequent-1-item.txt", "r")
file2 = open("/home/cloudera/illi/categories-drop-all-1-item.txt", "w")

for line in file1:
    if ";" in line:
        file2.write(line)

file1.close()
file2.close()

In [60]:
# remove lines from generated 2-items file if they are not in any actual transaction lines
# this could be skip as those 2-items will have zero support in next support calculation.

file1 = open("/home/cloudera/illi/categories-drop-all-1-item.txt", "r")
file2 = open("/home/cloudera/illi/part2-two-items.txt", "r")
file3 = open("/home/cloudera/illi/part2-two-items2.txt", "w")

for line2 in file2:
    item1,item2 = line2.split(";")
    item2 = item2[:-1]
    
    file1.seek(0, 0)
    for line1 in file1:
        if (item1 in line1) and (item2 in line1):
            file3.write(line2)
            break

file1.close()
file2.close()
file3.close()

In [61]:
# now calculate support for the 2-item categories

file1 = open("/home/cloudera/illi/categories-drop-all-1-item.txt", "r")
file2 = open("/home/cloudera/illi/part2-two-items2.txt", "r")
file3 = open("/home/cloudera/illi/part2-two-items-final.txt", "w")
file4 = open("/home/cloudera/illi/part2-two-items-infrequent.txt", "w")

for line2 in file2:
    item1,item2 = line2.split(";")
    item2 = item2[:-1]
    
    supp = 0
    file1.seek(0, 0)
    for line1 in file1:
        if (item1 in line1) and (item2 in line1):
            supp = supp + 1
    
    if supp >= 771:
        file3.write("%s:%s" % (supp,line2))
    else:
        file4.write("%s:%s" % (supp,line2))

file1.close()
file2.close()
file3.close()
file4.close()

In [62]:
# now we can work on 3-items categories
# generate 3-item categories

file1 = open("/home/cloudera/illi/part2-two-items-final.txt", "r")
file2 = open("/home/cloudera/illi/part2-three-items.txt", "w")

items_list = []
for line in file1:
    supp,items = line.split(":")
    item1,item2 = items.split(";")
    item2 = item2[:-1]
    if item1 not in items_list:
        items_list.append(item1)
    if item2 not in items_list:
        items_list.append(item2)
    
#print(len(items_list))
#print(";".join(items_list))

import itertools
for subset in itertools.combinations(items_list, 3):
    file2.write("%s;%s;%s\n" % (subset[0], subset[1], subset[2]))

file1.close()
file2.close()

In [63]:
# Transactions: 
# 1. delete 2-items transactions as we are working on three
# 2. delete transactions that include infrequent 2-items

file1 = open("/home/cloudera/illi/categories-drop-all-1-item.txt", "r")
file2 = open("/home/cloudera/illi/categories-drop-all-2-item.txt", "w")

for line in file1:
    if line.count(';') > 1:
        file2.write(line)


file1.close()
file2.close()

file3 = open("/home/cloudera/illi/part2-two-items-infrequent.txt", "r")
file4 = open("/home/cloudera/illi/categories-drop-all-2-item.txt", "r")
file5 = open("/home/cloudera/illi/categories-drop-all-2-item2.txt", "w")

for line4 in file4:
    flag = 0
    file3.seek(0, 0)
    for line3 in file3:
        supp,items = line3.split(":")
        item1,item2 = items.split(";")
        item2 = item2[:-1]
        if (item1 in line4) and (item2 in line4):
            flag = 1
            break
    if flag == 0:
        file5.write(line4)

file3.close()
file4.close()
file5.close()

In [64]:
# 3-item categories
# remove that are not in any transactions

file1 = open("/home/cloudera/illi/categories-drop-all-2-item2.txt", "r")
file2 = open("/home/cloudera/illi/part2-three-items.txt", "r")
file3 = open("/home/cloudera/illi/part2-three-items2.txt", "w")


for line2 in file2:
    item1,item2,item3 = line2.split(";")
    item3 = item3[:-1]
    
    file1.seek(0, 0)
    for line1 in file1:
        if (item1 in line1) and (item2 in line1) and (item3 in line1):
            file3.write(line2)
            break

file1.close()
file2.close()
file3.close()

In [65]:
# calculate support for 3-item categories
file1 = open("/home/cloudera/illi/categories-drop-all-2-item2.txt", "r")
file2 = open("/home/cloudera/illi/part2-three-items2.txt", "r")
file3 = open("/home/cloudera/illi/part2-three-items-final.txt", "w")
file4 = open("/home/cloudera/illi/part2-three-items-infrequent.txt", "w")

for line2 in file2:
    item1,item2,item3 = line2.split(";")
    item3 = item3[:-1]
    
    supp = 0
    file1.seek(0, 0)
    for line1 in file1:
        if (item1 in line1) and (item2 in line1) and (item3 in line1):
            supp = supp + 1
    
    if supp >= 771:
        file3.write("%s:%s" % (supp,line2))
    else:
        file4.write("%s:%s" % (supp,line2))

file1.close()
file2.close()
file3.close()
file4.close()


In [66]:
# 4- items generation

file1 = open("/home/cloudera/illi/part2-three-items-final.txt", "r")
file2 = open("/home/cloudera/illi/part2-four-items.txt", "w")

items_list = []
for line in file1:
    supp,items = line.split(":")
    item1,item2,item3 = items.split(";")
    item3 = item3[:-1]
    if item1 not in items_list:
        items_list.append(item1)
    if item2 not in items_list:
        items_list.append(item2)
    if item3 not in items_list:
        items_list.append(item3)
    
print(len(items_list))
print(";".join(items_list))

import itertools
for subset in itertools.combinations(items_list, 4):
    file2.write("%s;%s;%s;%s\n" % (subset[0], subset[1], subset[2], subset[3]))

file1.close()
file2.close()

3
Hotels & Travel;Hotels;Event Planning & Services


In [68]:
# so there is no four frequent item transactions 
# as the three frequent items only have three different items
# Hotels & Travel,   Hotels,   Event Planning & Services
# So there is no way to create four item combination.

# so next we can join 1-item frequent, 2-item frequent and 3-item frequent to be
# one file for part 2 submission.
# part1.txt
# part2-two-items-final.txt
# part2-three-items-final.txt

filenames = ['/home/cloudera/illi/part1.txt', '/home/cloudera/illi/part2-two-items-final.txt', 
             '/home/cloudera/illi/part2-three-items-final.txt']
with open('/home/cloudera/illi/part2-final.txt', 'w') as outfile:
    for fname in filenames:
        with open(fname) as infile:
            for line in infile:
                outfile.write(line)